In [48]:
import autosklearn.classification
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
import h2o
from h2o.automl import H2OAutoML
from tpot import TPOTClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report
import time

In [49]:
"""
This function initialises the auto sklearn framework and fits the training data to it. the best pipeine model is
then saved into a text file. In the repo the model files are saved succeeded by their dataset name. The sprint statistics
of the model is also displayed and saved into the file.
The predictions and prediction probabilities against the test data is then retunred back.
"""
def autosklearn_classification(X_train, y_train, X_test):
    
    #time_left_for_this_task=3600, per_run_time_limit=360. Use these arguments for a faster less accurate search
    automl = autosklearn.classification.AutoSklearnClassifier() 
    automl.fit(X_train, y_train)
    predictions = automl.predict(X_test)
    predictions_proba = automl.predict_proba(X_test)[:,1]
    text_file = open("models_tried.txt", "w+")
    n = text_file.write(str(automl.show_models()))
    n = text_file.write(str(automl.sprint_statistics()))
    text_file.close()
    print(automl.sprint_statistics())
    return (predictions,predictions_proba)
    

In [50]:
"""
This function initialises the tpot framework and fits the training data to it. The arguments are chosen so ask to return 
results dont take too long to train to make it coparable to other frameworks. The best pipeine model is
displayed to the user. 
it has the algorithm to be used with the necessary hyperparameters.
The predictions and prediction probabilities against the test data is then returned back.
"""
def tpot_classification(X_train, y_train, X_test):
    
    tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=1)
    tpot.fit(X_train, y_train)
    predictions = tpot.predict(X_test)
    predictions_proba = tpot.predict_proba(X_test)[:,1]
    return (predictions, predictions_proba)


In [51]:
"""
This function initialises the h2O framework and fits the training data to it. The arguments are chosen so ask to return 
results dont take too long to train to make it coparable to other frameworks. 
The top 10 leaderboard models is displayed to the user with the leader model used as default dueing predictions.
The predictions and prediction probabilities against the test data is then returned back.
"""
def h2o_classifications(X_train, y_train, X_test, target):
    
    h2o.init()
    aml = H2OAutoML()
    train_hf = pd.concat([X_train, y_train], axis=1)
    train_hf = h2o.H2OFrame(train_hf)
    train_hf[target] = train_hf[target].asfactor()
    aml.train(y = target, training_frame = train_hf)
    response = aml.predict(h2o.H2OFrame(X_test))
    print("various models tested:")
    lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
    print(lb)
    
    return response

In [98]:
"""
Function to calculate log loss and accuracy. Also returns the classification report containing precision, recall, f1 score
"""
def metric_calculator(framework, dataset, y_test, predictions):
    
    if framework =="H2O":   
        print("Framework: ", framework)
        ll = log_loss(y_test, predictions.as_data_frame().iloc[:,1:])
        print("log loss: ",ll)
        accuracy = sklearn.metrics.accuracy_score(predictions.as_data_frame().iloc[:,0], y_test.astype('int64'))
        print("accuracy: ",accuracy)
        print("Classification report")
        print(classification_report(y_test.astype('int64'), predictions.as_data_frame().iloc[:,0]))
    else:
        print("Framework: ", framework)
        ll = log_loss(y_test, predictions[1])
        print("log loss: ", ll)
        accuracy = sklearn.metrics.accuracy_score(predictions[0], y_test)
        print("accuracy: ",accuracy)
        print("Classification report")
        print(classification_report(y_test, predictions[0]))
            

### Dataset 1 - Wine quality dataset

In [95]:
ds1_df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",sep=";")

X, y = ds1_df.iloc[:,:-1], ds1_df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

<b> Model Auto-sklearn </b>

In [94]:
start_time = time.time()
ds1_autosklearn_predictions = autosklearn_classification(X_train, y_train, X_test)
end_time = time.time()


auto-sklearn results:
  Dataset name: 85c68cd19b887b3aa699aa9c7b59a288
  Metric: accuracy
  Best validation score: 0.659522
  Number of target algorithm runs: 242
  Number of successful target algorithm runs: 237
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0



In [16]:
#calculating metrics
metric_calculator("auto-sklearn", "wine quality", y_test, ds1_autosklearn_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  auto-sklearn
log loss:  0.4268143501
accuracy:  0.6522448979591837
Classification report
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.67      0.22      0.33        37
           5       0.70      0.61      0.65       368
           6       0.61      0.82      0.70       544
           7       0.76      0.47      0.58       233
           8       0.83      0.24      0.38        41

    accuracy                           0.65      1225
   macro avg       0.59      0.39      0.44      1225
weighted avg       0.67      0.65      0.64      1225

total time elapsed:  3602.586087703705


/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<b> Model TPOT </b>

In [96]:
start_time = time.time()
ds1_tpot_predictions = tpot_classification(X_train, y_train, X_test)


Generation 1 - Current best internal CV score: 0.6553237316725056

Generation 2 - Current best internal CV score: 0.6553237316725056

Generation 3 - Current best internal CV score: 0.6555928747520807

Generation 4 - Current best internal CV score: 0.6555928747520807

Generation 5 - Current best internal CV score: 0.6555928747520807

Best pipeline: KNeighborsClassifier(RobustScaler(input_matrix), n_neighbors=70, p=1, weights=distance)


In [97]:
end_time = time.time()

In [99]:
#calculating metrics
metric_calculator("tpot", "wine quality", y_test, ds1_tpot_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  tpot
log loss:  0.25763234
accuracy:  0.6644897959183673
Classification report


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       1.00      0.03      0.05        37
           5       0.73      0.60      0.66       368
           6       0.61      0.85      0.71       544
           7       0.76      0.50      0.60       233
           8       1.00      0.32      0.48        41

    accuracy                           0.66      1225
   macro avg       0.68      0.38      0.42      1225
weighted avg       0.70      0.66      0.65      1225

total time elapsed:  1908.7302021980286


<b> Model H2O </b>

In [61]:
start_time = time.time()
ds1_h2o_predictions = h2o_classifications(X_train, y_train, X_test, ds1_df.columns[-1])
end_time = time.time()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,16 hours 20 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,"28 days, 10 hours and 30 minutes"
H2O_cluster_name:,H2O_from_python_jupyter_htc3nt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.824 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
various models tested:


model_id,mean_per_class_error,logloss,rmse,mse,training_time_ms,predict_time_per_row_ms
GBM_grid__1_AutoML_20201216_020906_model_15,0.629892,0.974609,0.536878,0.288238,1341,0.145039
XGBoost_grid__1_AutoML_20201216_020906_model_5,0.630002,1.02758,0.540866,0.292536,927,0.026143
GBM_3_AutoML_20201216_020906,0.640258,0.936908,0.53719,0.288573,982,0.130451
GBM_grid__1_AutoML_20201216_020906_model_22,0.640654,0.966226,0.52975,0.280635,1623,0.221353
StackedEnsemble_BestOfFamily_AutoML_20201216_020906,0.641232,0.869254,0.536349,0.28767,21755,0.148778
XRT_1_AutoML_20201216_020906,0.642959,1.29546,0.544668,0.296664,1193,0.098868
DRF_1_AutoML_20201216_020906,0.643066,1.32428,0.544484,0.296463,735,0.086306
GBM_4_AutoML_20201216_020906,0.649275,0.934342,0.533898,0.285047,1887,0.131289
GBM_grid__1_AutoML_20201216_020906_model_17,0.649514,0.954645,0.541141,0.292834,1633,0.189937
GBM_grid__1_AutoML_20201216_020906_model_24,0.650336,0.956071,0.536499,0.287831,1413,0.181271


In [73]:
#calculating metrics
metric_calculator("H2O", "wine quality",y_test, ds1_h2o_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  H2O
log loss:  0.32154732
accuracy:  0.6587755102040816
Classification report
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         2
           4       0.53      0.22      0.31        37
           5       0.70      0.65      0.67       368
           6       0.64      0.78      0.70       544
           7       0.68      0.51      0.58       233
           8       0.64      0.34      0.44        41

    accuracy                           0.66      1225
   macro avg       0.53      0.42      0.45      1225
weighted avg       0.66      0.66      0.65      1225

total time elapsed:  2645.5268199443817


### Dataset 2 - Spambase


In [75]:
X, y = fetch_openml('spambase', version=1, return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

<b> Model Auto-sklearn </b>

In [1]:
start_time = time.time()
ds2_autosklearn_predictions = autosklearn_classification(X_train, y_train, X_test)
end_time = time.time()


auto-sklearn results:
  Dataset name: 2b36186d2a8d5021e8a046ef162a9f3e
  Metric: accuracy
  Best validation score: 0.955224
  Number of target algorithm runs: 138
  Number of successful target algorithm runs: 132
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0



In [13]:
#calculating metrics
metric_calculator("auto-sklearn", "spambase", y_test, ds2_autosklearn_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  auto-sklearn
log loss:  0.18817840144885561
accuracy:  0.9565595134665508
Classification report
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       701
           1       0.97      0.92      0.94       450

    accuracy                           0.96      1151
   macro avg       0.96      0.95      0.95      1151
weighted avg       0.96      0.96      0.96      1151

total time elapsed:  3624.878770828247


<b> Model TPOT </b>

In [76]:
start_time = time.time()
ds2_tpot_predictions = tpot_classification(X_train, y_train, X_test)
end_time = time.time()


Generation 1 - Current best internal CV score: 0.946376811594203

Generation 2 - Current best internal CV score: 0.9489855072463769

Generation 3 - Current best internal CV score: 0.9501449275362319

Generation 4 - Current best internal CV score: 0.9515942028985507

Generation 5 - Current best internal CV score: 0.952463768115942

Best pipeline: GradientBoostingClassifier(BernoulliNB(input_matrix, alpha=0.01, fit_prior=True), learning_rate=0.1, max_depth=6, max_features=0.5, min_samples_leaf=15, min_samples_split=5, n_estimators=100, subsample=0.7500000000000001)


In [77]:
#calculating metrics
metric_calculator("tpot", "spambase", y_test,ds2_tpot_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  tpot
log loss:  0.12303342451402823
accuracy:  0.9548218940052129
Classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       701
           1       0.95      0.94      0.94       450

    accuracy                           0.95      1151
   macro avg       0.95      0.95      0.95      1151
weighted avg       0.95      0.95      0.95      1151

total time elapsed:  1726.0997183322906


<b> Model H2O </b>

In [78]:
start_time = time.time()
ds2_h2o_predictions = h2o_classifications(X_train, y_train, X_test, y.name)
end_time = time.time()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,17 hours 43 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,"28 days, 11 hours and 53 minutes"
H2O_cluster_name:,H2O_from_python_jupyter_htc3nt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.581 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
various models tested:


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
GBM_grid__1_AutoML_20201216_033155_model_32,0.9883,0.130907,0.981757,0.0502501,0.190995,0.0364791,1880,0.04003
GBM_grid__1_AutoML_20201216_033155_model_7,0.988236,0.131039,0.982811,0.0500404,0.190789,0.0364005,1541,0.041045
GBM_grid__1_AutoML_20201216_033155_model_10,0.988045,0.13043,0.981372,0.0529375,0.19043,0.0362637,1434,0.038728
StackedEnsemble_AllModels_AutoML_20201216_033155,0.987873,0.172869,0.981719,0.0479368,0.199344,0.0397381,6028,1.18797
GBM_4_AutoML_20201216_033155,0.987828,0.133157,0.982363,0.0490074,0.192263,0.0369649,1806,0.03744
GBM_grid__1_AutoML_20201216_033155_model_5,0.987744,0.137402,0.982553,0.0544425,0.196351,0.0385538,2112,0.04198
GBM_grid__1_AutoML_20201216_033155_model_51,0.98774,0.132355,0.98164,0.0491424,0.192053,0.0368845,1585,0.045376
GBM_3_AutoML_20201216_033155,0.987647,0.133459,0.982328,0.0483562,0.19247,0.0370446,1733,0.038299
GBM_grid__1_AutoML_20201216_033155_model_31,0.98764,0.141162,0.98164,0.0568604,0.198781,0.039514,2404,0.044602
GBM_grid__1_AutoML_20201216_033155_model_44,0.987592,0.133015,0.982397,0.0496061,0.190011,0.0361041,1583,0.042892


In [79]:
#calculating metrics
metric_calculator("H2O", "spambase",y_test,ds2_h2o_predictions )
print("total time elapsed: ",end_time - start_time)

Framework:  H2O
log loss:  0.10893043400624643
accuracy:  0.9600347523892268
Classification report
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       701
           1       0.94      0.96      0.95       450

    accuracy                           0.96      1151
   macro avg       0.96      0.96      0.96      1151
weighted avg       0.96      0.96      0.96      1151

total time elapsed:  3086.9009573459625


### Dataset 3 - Bank Marketing

In [80]:
Marketing = pd.read_csv("Data/bank-additional-full.csv",sep=",")

X, y = Marketing.iloc[:,:-1], Marketing.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

<b> Model Auto-sklearn </b>

In [2]:
start_time = time.time()

for column in Marketing.columns:
    if str(Marketing[column].dtypes) =="object":
        Marketing[column] = Marketing[column].astype("category")
        
X, y = Marketing.iloc[:,:-1], Marketing.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

ds3_autosklearn_predictions = autosklearn_classification(X_train, y_train, X_test)
end_time = time.time()


Dataset name: e8a345c8c7e197380470c38aa26d24c8
  Metric: accuracy
  Best validation score: 0.917214
  Number of target algorithm runs: 48
  Number of successful target algorithm runs: 42
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0
      


In [10]:
#calculating metrics
metric_calculator("auto-sklearn", "bank marketing", y_test, ds3_autosklearn_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  auto-sklearn
log loss:  0.2102055772435655
accuracy:  0.9181314946100806
Classification report
              precision    recall  f1-score   support

          no       0.94      0.97      0.95      9132
         yes       0.68      0.52      0.59      1165

    accuracy                           0.92     10297
   macro avg       0.81      0.75      0.77     10297
weighted avg       0.91      0.92      0.91     10297

total time elapsed:  3601.192629814148


<b> Model TPOT </b>

In [82]:
start_time = time.time()

Marketing.rename(columns={'y': 'class'}, inplace=True)

<b>Preprocessing since TPOT accepts only numeric data in its features.</b>

In [83]:
# Lookign at number of categories in each feature
for cat in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome' ,'class']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, Marketing[cat].unique().size))

Number of levels in category 'job': 12.00 
Number of levels in category 'marital': 4.00 
Number of levels in category 'education': 8.00 
Number of levels in category 'default': 3.00 
Number of levels in category 'housing': 3.00 
Number of levels in category 'loan': 3.00 
Number of levels in category 'contact': 2.00 
Number of levels in category 'month': 10.00 
Number of levels in category 'day_of_week': 5.00 
Number of levels in category 'poutcome': 3.00 
Number of levels in category 'class': 2.00 


In [84]:
for cat in ['contact', 'poutcome','class', 'marital', 'default', 'housing', 'loan']:
    print("Levels for catgeory '{0}': {1}".format(cat, Marketing[cat].unique()))

Levels for catgeory 'contact': ['telephone', 'cellular']
Categories (2, object): ['telephone', 'cellular']
Levels for catgeory 'poutcome': ['nonexistent', 'failure', 'success']
Categories (3, object): ['nonexistent', 'failure', 'success']
Levels for catgeory 'class': ['no', 'yes']
Categories (2, object): ['no', 'yes']
Levels for catgeory 'marital': ['married', 'single', 'divorced', 'unknown']
Categories (4, object): ['married', 'single', 'divorced', 'unknown']
Levels for catgeory 'default': ['no', 'unknown', 'yes']
Categories (3, object): ['no', 'unknown', 'yes']
Levels for catgeory 'housing': ['no', 'yes', 'unknown']
Categories (3, object): ['no', 'yes', 'unknown']
Levels for catgeory 'loan': ['no', 'yes', 'unknown']
Categories (3, object): ['no', 'yes', 'unknown']


In [85]:
Marketing['marital'] = Marketing['marital'].map({'married':0,'single':1,'divorced':2,'unknown':3})
Marketing['default'] = Marketing['default'].map({'no':0,'yes':1,'unknown':2})
Marketing['housing'] = Marketing['housing'].map({'no':0,'yes':1,'unknown':2})
Marketing['loan'] = Marketing['loan'].map({'no':0,'yes':1,'unknown':2})
Marketing['contact'] = Marketing['contact'].map({'telephone':0,'cellular':1})
Marketing['poutcome'] = Marketing['poutcome'].map({'nonexistent':0,'failure':1,'success':2})
Marketing['class'] = Marketing['class'].map({'no':0,'yes':1})

In [86]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

job_Trans = mlb.fit_transform([{str(val)} for val in Marketing['job'].values])
education_Trans = mlb.fit_transform([{str(val)} for val in Marketing['education'].values])
month_Trans = mlb.fit_transform([{str(val)} for val in Marketing['month'].values])
day_of_week_Trans = mlb.fit_transform([{str(val)} for val in Marketing['day_of_week'].values])

In [87]:
#Droping features with too many categories since this will reduce training accuracy

marketing_new = Marketing.drop(['marital','default','housing','loan','contact','poutcome','class','job','education','month','day_of_week'], axis=1)
marketing_new = np.hstack((marketing_new.values, job_Trans, education_Trans, month_Trans, day_of_week_Trans))

In [88]:
X, y = marketing_new[:,:-1], marketing_new[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [89]:
ds3_tpot_predictions = tpot_classification(X_train, y_train, X_test)
end_time = time.time()


Generation 1 - Current best internal CV score: 1.0

Generation 2 - Current best internal CV score: 1.0

Generation 3 - Current best internal CV score: 1.0

Generation 4 - Current best internal CV score: 1.0

Generation 5 - Current best internal CV score: 1.0

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=6, min_samples_leaf=20, min_samples_split=14)


In [90]:
metric_calculator("auto-sklearn", "spambase", y_test, ds3_tpot_predictions)
print("total time elapsed: ",end_time - start_time)

Framework:  auto-sklearn
log loss:  9.992007221626413e-16
accuracy:  1.0
Classification report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      8233
         1.0       1.00      1.00      1.00      2064

    accuracy                           1.00     10297
   macro avg       1.00      1.00      1.00     10297
weighted avg       1.00      1.00      1.00     10297

total time elapsed:  7903.794376850128


<b> Model H2O </b>

In [92]:
X, y = Marketing.iloc[:,:-1], Marketing.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

start_time = time.time()
ds3_h2o_predictions = h2o_classifications(X_train, y_train, X_test, y.name)
end_time = time.time()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,21 hours 18 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,"28 days, 15 hours and 28 minutes"
H2O_cluster_name:,H2O_from_python_jupyter_htc3nt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.330 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
various models tested:


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_BestOfFamily_AutoML_20201216_070656,0.948524,0.192412,0.664305,0.141878,0.241443,0.0582947,1592,0.065912
XGBoost_grid__1_AutoML_20201216_070656_model_11,0.948425,0.173877,0.664004,0.138118,0.23636,0.0558661,2425,0.007776
XGBoost_grid__1_AutoML_20201216_070656_model_2,0.948266,0.174288,0.661531,0.135707,0.236591,0.0559752,3073,0.009556
XGBoost_grid__1_AutoML_20201216_070656_model_21,0.948066,0.174754,0.661083,0.146589,0.236684,0.0560194,3363,0.009566
XGBoost_grid__1_AutoML_20201216_070656_model_63,0.948049,0.174699,0.657175,0.147311,0.237278,0.0563008,2407,0.006565
XGBoost_grid__1_AutoML_20201216_070656_model_16,0.948009,0.175887,0.650429,0.152539,0.238355,0.0568132,3064,0.007677
GBM_grid__1_AutoML_20201216_070656_model_18,0.94791,0.175662,0.653219,0.149311,0.237103,0.056218,1870,0.022674
XGBoost_grid__1_AutoML_20201216_070656_model_25,0.947756,0.175882,0.651054,0.147457,0.237977,0.0566329,1965,0.005247
XGBoost_grid__1_AutoML_20201216_070656_model_6,0.9477,0.176839,0.647576,0.14819,0.239204,0.0572185,2906,0.006634
XGBoost_grid__1_AutoML_20201216_070656_model_29,0.947623,0.175347,0.658542,0.14786,0.237398,0.0563579,1948,0.005819


In [93]:
#Calulating metrics
metric_calculator("H2O", "bank-marketing",y_test,ds3_h2o_predictions )
print("total time elapsed: ",end_time - start_time)

Framework:  H2O
log loss:  0.1904871474818192
accuracy:  0.9090026221229485
Classification report
              precision    recall  f1-score   support

           0       0.97      0.92      0.95      9132
           1       0.57      0.80      0.66      1165

    accuracy                           0.91     10297
   macro avg       0.77      0.86      0.81     10297
weighted avg       0.93      0.91      0.92     10297

total time elapsed:  3048.9236533641815
